### Article this is derived from : http://mnemstudio.org/path-finding-q-learning-tutorial.htm

### Load Libs

In [1]:
import numpy as np
import pylab as plt
import random

### Implement a simple Q Learner

In [2]:
# List of points that are mapped to each other. 
# Our goal is to get from any point to the goal point, in this case 5

environment = [(0,4),(4,3),(2,3),(1,3),(4,5),(1,5)]

In [3]:
## Rewards Matrix 

# We have 6 possible states and 6 possible actions
# We have 6 actions because, lets say from 3, we can choose an action to go to state 2, 1 or 4. 
# Lets look at it another way, we have the following possible actions
# Up, Down, Front, back, slant in, slant out ( Ok bad choice of names, but I am creatively impaired, sooo ...)
states = 6
actions = 6
goal = 5

R = np.matrix(np.ones(shape=[states,actions]))
R = R * -1 # We assign -1 to every value

for row in environment:
    R[row] = 0
    if row[1] == goal:
        R[row] = 100
    row = row[::-1]
    R[row] = 0

R[goal,goal] = 100

In [4]:
## Q matrix : Our brain
# Initially, we will have an empty Q matrix, which as we iterate through the different possible states and actions
# the RL process will populate with values it can use to make decisions. 

Q = np.matrix(np.zeros(shape=[MATRIX_SIZE,MATRIX_SIZE]))

### Lets look at Rewards and Q Table

In [5]:
print("The rewards Matrix is \n \n {}".format(R))
print("\n ")
print("The Q table Matrix is \n \n {}".format(Q))


The rewards Matrix is 
 
 [[ -1.  -1.  -1.  -1.   0.  -1.]
 [ -1.  -1.  -1.   0.  -1. 100.]
 [ -1.  -1.  -1.   0.  -1.  -1.]
 [ -1.   0.   0.  -1.   0.  -1.]
 [  0.  -1.  -1.   0.  -1. 100.]
 [ -1.   0.  -1.  -1.   0. 100.]]

 
The Q table Matrix is 
 
 [[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [6]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "http://mnemstudio.org/ai/path/images/map3a.gif")

In [7]:
# Now let us create the structure of the simple Q Learner
print("1) Please refer to the image above for better clarity \n")

# Current State
currentstate = 1

# Let us get all possible actions for current state
# From the rewards function, we can see which direction we can go in 

def available_actions(state):
    rewards_row_state = R[state,]
    av_actions = np.where(rewards_row_state >= 0)[1] # In our case, any value ge 0 is a possible action we can take from our current state
    return av_actions
    
actions = available_actions(currentstate)
print("2) As we can see, the available actions from state {} is {} \n".format(current_state,actions))

# Lets say, we choose one action from the available actions, so we choose randomly

def choose_random_action(actions):
    action = random.choice(actions)
    return action

action = choose_random_action(actions)
print("3) The action we have chosen to take is to go to {}, from state {} \n".format(action,current_state))

# we now have information about our current state, current action we are taking, 
# and possible next states and actions we can take

gamma = 0.8

print("4) Here we have pause a little and introduce a hyper parameter called Gamma")
print("   Gamma is a paramter that is used to control if the model needs to focus on immediate reward or future rewards \n")

# Now we develop our Q update function

print("5) The Equation for our Q learner is as follows")
print("   Q[state,action] = R[state,action] + gamma * Max(Q[next state,all_actions])")

def Q_update(state,action,gamma):
    # First the max Q part
    Max_Q = np.max(Q[action,])
    Q[state,action] = R[state,action] + gamma * Max_Q
    return Q

Q = Q_update(currentstate,action,gamma)



1) Please refer to the image above for better clarity 

2) As we can see, the available actions from state 1 is [3 5] 

3) The action we have chosen to take is to go to 3, from state 1 

4) Here we have pause a little and introduce a hyper parameter called Gamma
   Gamma is a paramter that is used to control if the model needs to focus on immediate reward or future rewards 

5) The Equation for our Q learner is as follows
   Q[state,action] = R[state,action] + gamma * Max(Q[next state,all_actions])


### Now we create a Monte Carlo simulation of the Markov Decision Process to converge this Q matrix (Our Brain)

In [17]:
for i in range(100):
    current_state = random.choice([0,1,2,3,4,5]) # Step 1
    actions = available_actions(current_state) # Step 2
    action = choose_random_action(actions) # Step 3
    Q = Q_update(current_state,action,gamma) # Step 4
    
# We can normalize Q Matrix values for a better idea of what is going on

Q = Q/np.max(Q)*100

In [18]:
Q

matrix([[  0.        ,   0.        ,   0.        ,   0.        ,
          76.54487665,   0.        ],
        [  0.        ,   0.        ,   0.        ,  61.23590132,
           0.        , 100.        ],
        [  0.        ,   0.        ,   0.        ,  61.23590132,
           0.        ,   0.        ],
        [  0.        ,  75.08183323,  48.98872106,   0.        ,
          76.54487665,   0.        ],
        [ 61.23590132,   0.        ,   0.        ,  61.23590132,
           0.        ,  95.68109582],
        [  0.        ,  75.08183323,   0.        ,   0.        ,
          76.54487665,  99.25092177]])

### Lets take a look at the Image again

In [9]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "http://mnemstudio.org/ai/path/images/map3a.gif")

In [10]:
currentstate = 4
tracking = []
tracking.append(currentstate)

while currentstate != goal:
    nextstep = np.where(Q[currentstate,] == np.max(Q[currentstate,]))[1]
    tracking.append(nextstep[0])
    
    if len(nextstep) > 1:
        nextstep = random.choice(nextstep)
    else:
        nextstep = nextstep
    
    currentstate = nextstep
        
print("The most efficient path from state {} is {}".format(step[0],step))

The most efficient path from state 4 is [4, 5]
